#### Semantic kernel - Plugins example

In [104]:
import asyncio
from typing import Annotated
import dotenv
import os
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings import (
    OpenAIChatPromptExecutionSettings,
)
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.kernel import Kernel

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel


In [105]:
dotenv.load_dotenv("..\\common\\credentials.env", verbose=True, override=True)

True

In [ ]:
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
aoai_api_type = os.environ["OPENAI_API_TYPE"]
deployment_name = "gpt-4o" # #use latest model like 4o or similar. GPT 3.5 turbo or older models doesnt seem to work as function calling/parallel tools calls is not supported in those models
aoai_api_version = "2024-08-01-preview" #Using latest version as of this date.
# Note:  Older versions of APIs than Aug 2024 and  old models doesn't work with function calling/parallel tools calls throwing error 
# as Unrecognized request argument supplied: parallel_tool_calls', 'type': 'invalid_request_error'

#### Example 1: Weather and Time Assistant

In [118]:
class WeatherPlugin:
    """A sample plugin that provides weather information for cities."""

    @kernel_function(name="get_weather_for_city", description="Get the weather for a city")
    def get_weather_for_city(self, city: Annotated[str, "The input city"]) -> Annotated[str, "The output is a string"]:
        if city == "Boston":
            return "61 and rainy"
        if city == "London":
            return "55 and cloudy"
        if city == "Miami":
            return "80 and sunny"
        if city == "Paris":
            return "60 and rainy"
        if city == "Tokyo":
            return "50 and sunny"
        if city == "Sydney":
            return "75 and sunny"
        if city == "Tel Aviv":
            return "80 and sunny"
        return "31 and snowing"
    


Let us use above WeatherPlugin to demonstrate the Plugin feature of SK. For getting current time we will use SK's readymade plugin called as TimePlugin 

In [ ]:
#async def main():
kernel = Kernel()

service_id = "function_calling"

# LLM
ai_service = AzureChatCompletion(
    service_id=service_id,api_key=aoai_api_key, endpoint=aoai_endpoint, api_version=aoai_api_version,deployment_name=deployment_name
)

kernel.add_service(ai_service)
kernel.add_plugin(TimePlugin(), plugin_name="time")
kernel.add_plugin(WeatherPlugin(), plugin_name="weather")

# automated function calling with a non-streaming prompt
print("========== Use automated function calling with a non-streaming prompt ==========")
settings: OpenAIChatPromptExecutionSettings = kernel.get_prompt_execution_settings_from_service_id(
    service_id=service_id
)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto(filters={"included_plugins": ["weather", "time"]})

print(
    await kernel.invoke_prompt(        
        prompt="""Given the current time of day and weather, what is the likely color of the sky in Boston? How much is temperature there?
        Make sure you convert current time from US/Central timezone to different timezone based on city name provided.
        """,
        settings=settings,
    )
)

========== Use automated function calling with a non-streaming prompt ==========
The current time in US/Central timezone is 6:47 PM. The current weather in Boston is 61°F and rainy.

Now, let's convert the current time from US/Central to US/Eastern timezone. The time difference between Central Time (CT) and Eastern Time (ET) is +1 hour. Thus, the time in Boston would be 7:47 PM.

Given the time and weather conditions:
- At 7:47 PM, it would be considered after sunset, so the sky is likely to be dark.
- The current temperature is 61°F. 

Therefore, the sky in Boston is most likely dark due to it being evening and rainy. The temperature is 61°F.


#### Example 2: Restaurant menu

In [ ]:
# Define a sample plugin for the sample
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Broccoli cheddar
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"

In [143]:
kernel = Kernel()

# Add the AzureChatCompletion AI Service to the Kernel
service_id = "agent"    

# Please make sure your AzureOpenAI Deployment allows for function calling
ai_service = AzureChatCompletion(
    service_id=service_id,api_key=aoai_api_key, endpoint=aoai_endpoint, api_version=aoai_api_version,deployment_name=deployment_name
)    
kernel.add_service(ai_service)

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Add the MenuPlugin to the Kernel
kernel.add_plugin(MenuPlugin(), plugin_name="menu")

# Create the agent
agent = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="SampleAssistantAgent",
    instructions=f"""
        You are an agent designed to help with the restaurant menu order.
        """,
    execution_settings=settings
    )

history = ChatHistory()
is_complete: bool = True


In [146]:
user_input = "what is the special menu today?"

history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

async for response in agent.invoke(history=history):
            print(f"{response.content}")

The special menu today includes:
- **Special Soup**: Clam Chowder
- **Special Salad**: Cobb Salad
- **Special Drink**: Chai Tea


In [147]:
user_input = "can you share price for the menu?"

history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

async for response in agent.invoke(history=history):
            print(f"{response.content}")

The special menu items for today and their prices are:

- **Special Soup: Clam Chowder** - $9.99
- **Special Salad: Cobb Salad** - $9.99
- **Special Drink: Chai Tea** - $9.99
